In [4]:
#from fbprophet import Prophet
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

ModuleNotFoundError: No module named 'prophet'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
path = os.getcwd()

path

## Dataframe

In [ ]:
df = pd.read_csv("../DF/grand_master_report.csv")

In [ ]:
df.head()

In [ ]:
manufacturer_columns=[col for col in df.columns.to_list() if col.endswith('Manufacturer')]
df.drop(columns=manufacturer_columns,inplace=True)
df

In [ ]:
df['Day\nDate'] = pd.to_datetime(df['Day\nDate'], format='%d.%m.%y')

In [ ]:
df['m_y'] = df['Day\nDate'].dt.strftime('%m.%Y')

In [ ]:
df['Day\nDate'] = pd.to_datetime(df['Day\nDate'], format='%d.%m.%y')

In [ ]:
df

In [ ]:
df["month_year"] = df['Day\nDate'].dt.to_period('M')


In [ ]:
df["year"] = df['Day\nDate'].dt.to_period('Y')

In [ ]:
df = df.sort_values(by='Day\nDate', ascending=True)
df

In [ ]:
df['mm'] = pd.to_datetime(df['Day\nDate']).dt.month
df

In [ ]:
df.info()

## Visualisation

In [ ]:
plt.figure(figsize=(28, 6))

sns.lineplot(data=df, x="Day\nDate", y="NewPayingUsers")
sns.lineplot(data=df, x="Day\nDate", y="PaidMeals")


In [ ]:
plt.figure(figsize=(28, 6))

sns.lineplot(data=df, x="Day\nDate", y="AddedSupply")
sns.lineplot(data=df, x="Day\nDate", y="SubtractedSupply")

In [ ]:

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="Day\nDate", y="ActiveUsers")

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="Day\nDate", y="DeclaredSupply")

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="m_y", y="DeclaredSupply")

In [ ]:
df['month_year'].value_counts()

In [ ]:
plt.figure(figsize=(28, 6))

sns.lineplot(data=df, x="Day\nDate", y="ActiveUsers")

In [ ]:
plt.figure(figsize=(28,15))
sns.heatmap(df.corr(), cmap='coolwarm', annot = True, annot_kws={"size": 12})

In [ ]:
plt.figure(figsize=(28, 15))

sns.lineplot(data=df, x="m_y", y="ActiveUsers")
sns.lineplot(data=df, x="m_y", y="MealsSaved")


In [ ]:
plt.figure(figsize=(28, 10))

sns.scatterplot(data=df, x="PayingUsers", y="MealsSaved", hue="MealsSaved")

In [ ]:
plt.figure(figsize=(28, 10))

sns.scatterplot(data=df, x="NewPayingUsers", y="MealsSaved", hue="MealsSaved", sizes=(200, 80))

In [ ]:
plt.figure(figsize=(28, 10))

sns.scatterplot(data=df, x="ActiveUsers", y="PayingUsers", hue="All-TimePayingUsers")

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(28, 6))


sns.boxplot(data=df, x="m_y", y="ActiveUsers")


In [ ]:
df.head()

## Prophet MealsSaved




In [ ]:
dfp = pd.DataFrame(df, columns = ['Day\nDate', 'MealsSaved'])
dfp

In [ ]:
dfp.columns = ['ds', 'y']

dfp.head()

In [ ]:
dfp['ds'] = pd.to_datetime(dfp['ds'])
dfp

In [ ]:
plt.figure(figsize=(12, 8))
plt.grid(linestyle='-.')

plt.plot(dfp['ds'], dfp['y'], 'b')
plt.show()

In [ ]:
train = dfp.iloc[:440]
test = dfp.iloc[440:]

In [ ]:
train.shape, test.shape

In [ ]:
train

# Model

In [ ]:
model = Prophet(seasonality_mode='multiplicative', interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True)
model.fit(train)

In [ ]:
horizon = 36
future = model.make_future_dataframe(periods=horizon)
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
ax = model.plot(forecast);

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(train['ds'], train['y'], label="Train")
plt.plot(test['ds'], test['y'], label="Test")
plt.plot(forecast[-horizon:]['ds'], forecast[-horizon:]['yhat'], label="predictions")
plt.legend(loc="best")
plt.show()

In [ ]:
model.plot_components(forecast);

In [ ]:
800 days total dataset
initial = 624 == size of train dataset
horizon = 20 == size to be forecasted
period = 10 (default = 1/2 of horizon) == spacing between cutoff dates as incremental

440
initial = 440
horizon = 20
period = 10

In [ ]:
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

cv_results = cross_validation(model=model, initial="343 days", horizon='30 days', period='7 days')

In [ ]:
cv_results

In [ ]:
dfp_p = performance_metrics(cv_results)
dfp_p

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
plot_cross_validation_metric(cv_results, metric='mse');

## Prophet NewRegisteredUsers

### Df

In [ ]:
dfp_nu = pd.DataFrame(df, columns = ['Day\nDate', 'NewRegisteredUsers'])
dfp_nu

In [ ]:
dfp_nu.columns = ['ds', 'y']

dfp_nu.head()

In [ ]:
dfp_nu['ds'] = pd.to_datetime(dfp_nu['ds'])
dfp_nu

In [2]:
plt.figure(figsize=(12, 8))
plt.grid(linestyle='-.')

plt.plot(dfp_nu['ds'], dfp_nu['y'], 'b')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
train_nu = dfp_nu.iloc[:350]
test_nu = dfp_nu.iloc[350:]

In [ ]:
train_nu.shape, test_nu.shape

In [ ]:
model_nu = Prophet(seasonality_mode='multiplicative', interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True)
model_nu.fit(train_nu)

In [ ]:
horizon = 24
future = model_nu.make_future_dataframe(periods=horizon)
forecast = model_nu.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
ax = model_nu.plot(forecast);

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(train_nu['ds'], train_nu['y'], label="Train")
plt.plot(test_nu['ds'], test_nu['y'], label="Test")
plt.plot(forecast[-horizon:]['ds'], forecast[-horizon:]['yhat'], label="predictions")
plt.legend(loc="best")
plt.show()

In [ ]:
model_nu.plot_components(forecast);

In [ ]:
m = Prophet()
m.fit(dfp_nu)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast

In [ ]:
import numpy as np
f_yhat = forecast["yhat"]
f_AU = f_yhat[350:500]


e_y = dfp_nu["y"]
e_AU = e_y[350:]


In [ ]:
e_AU = e_AU.astype("float")

In [ ]:
e_AU = np.array(e_AU.values.tolist())

In [ ]:
expected_AU = e_AU
expected_AU

In [ ]:
f_AU = np.array(f_AU.values.tolist())

In [ ]:
forecast_AU = f_AU
forecast_AU

In [ ]:
#np array beginning +350 days after 
#np array slize 150

In [ ]:
fig1 = m.plot(forecast)


In [ ]:
fig2 = m.plot_components(forecast)
fig2

### Metrics

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast_AU - expected_AU)/np.abs(expected_AU))  # MAPE
    me = np.mean(forecast_AU - expected_AU)             # ME
    mae = np.mean(np.abs(forecast_AU - expected_AU))    # MAE
    mpe = np.mean((forecast_AU - expected_AU)/expected_AU)   # MPE
    rmse = np.mean((forecast_AU - expected_AU)**2)**.5  # RMSE
    corr = np.corrcoef(forecast_AU, expected_AU)[0,1]   # corr
    mins = np.amin(np.hstack([forecast_AU[:,None], 
                              expected_AU[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast_AU[:,None], 
                              expected_AU[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax                     
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(forecast_AU, expected_AU)

In [ ]:
#yhat - predicted variable
#y - dependent variable 

### Cross_validation

In [ ]:
cv_results_nu = cross_validation(model=model_nu, initial="150 days", horizon='60 days', period='30 days')

In [ ]:
cv_results_nu

In [ ]:
dfp_nu = performance_metrics(cv_results_nu)
dfp_nu

In [ ]:
plot_cross_validation_metric(cv_results_nu, metric='mse');

## Prophet PayingUsers

In [ ]:
dfp_pu = pd.DataFrame(df, columns = ['Day\nDate', 'PayingUsers'])
dfp_pu

In [ ]:
dfp_pu.columns = ['ds', 'y']
dfp_pu.head()

In [ ]:
dfp_pu['ds'] = pd.to_datetime(dfp_pu['ds'])
dfp_pu

In [ ]:
plt.figure(figsize=(12, 8))
plt.grid(linestyle='-.')

plt.plot(dfp_pu['ds'], dfp_pu['y'], 'b')
plt.show()

In [ ]:
train_pu = dfp_pu.iloc[:350]
test_pu = dfp_pu.iloc[350:]

In [ ]:
train_pu.shape, test_pu.shape

In [ ]:
model_pu = Prophet(seasonality_mode='multiplicative', interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True)
model_pu.fit(train_pu)


In [ ]:
horizon = 24
future = model_pu.make_future_dataframe(periods=horizon)
forecast = model_pu.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
ax = model_pu.plot(forecast_pu);

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(train_pu['ds'], train_pu['y'], label="Train")
plt.plot(test_pu['ds'], test_pu['y'], label="Test")
plt.plot(forecast[-horizon:]['ds'], forecast[-horizon:]['yhat'], label="predictions")
plt.legend(loc="best")
plt.show()

In [ ]:
model_pu.plot_components(forecast);

In [ ]:
m_pu = Prophet()
m_pu.fit(dfp_pu)

In [ ]:
future_pu = m_pu.make_future_dataframe(periods=365)
future_pu.tail()

In [ ]:
forecast_pu = m_pu.predict(future_pu)
forecast_pu[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast_pu

In [ ]:
import numpy as np
f_yhat_pu = forecast_pu["yhat"]
f_AU_pu = f_yhat_pu[350:500]


e_y_pu = dfp_pu["y"]
e_AU_pu = e_y_pu[350:]



In [ ]:
e_AU_pu = np.array(e_AU_pu.values.tolist())


In [ ]:
expected_AU_pu = e_AU_pu
expected_AU_pu

In [ ]:
f_AU_pu = np.array(f_AU_pu.values.tolist())


In [ ]:
forecast_AU_pu = f_AU_pu
forecast_AU_pu

In [ ]:
fig2 = m_pu.plot(forecast_pu);


### Metrics PayingUsers

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast_AU_pu - expected_AU_pu)/np.abs(expected_AU_pu))  # MAPE
    me = np.mean(forecast_AU_pu - expected_AU_pu)             # ME
    mae = np.mean(np.abs(forecast_AU_pu - expected_AU_pu))    # MAE
    mpe = np.mean((forecast_AU_pu - expected_AU_pu)/expected_AU_pu)   # MPE
    rmse = np.mean((forecast_AU_pu - expected_AU_pu)**2)**.5  # RMSE
    corr = np.corrcoef(forecast_AU_pu, expected_AU_pu)[0,1]   # corr
    mins = np.amin(np.hstack([forecast_AU_pu[:,None], 
                              expected_AU_pu[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast_AU_pu[:,None], 
                              expected_AU_pu[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax                     
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(forecast_AU_pu, expected_AU_pu)

### Cross_validation

In [ ]:
cv_results_pu = cross_validation(model=model_pu, initial="343 days", horizon='30 days', period='7 days')

In [ ]:
cv_results_pu

In [3]:
dfp_pu = performance_metrics(cv_results_pu)
dfp_pu

NameError: name 'performance_metrics' is not defined

In [ ]:
plot_cross_validation_metric(cv_results_pu, metric='mse');

## Prophet ActiveUsers

In [ ]:
dfp_au = pd.DataFrame(df, columns = ['Day\nDate', 'ActiveUsers'])
dfp_au

In [ ]:
dfp_au.columns = ['ds', 'y']
dfp_au.head()

In [ ]:
dfp_au['ds'] = pd.to_datetime(dfp['ds'])
dfp_au

In [ ]:
plt.figure(figsize=(12, 8))
plt.grid(linestyle='-.')

plt.plot(dfp_au['ds'], dfp_au['y'], 'b')
plt.show()

In [ ]:
train_au = dfp_au.iloc[:350]
test_au = dfp_au.iloc[350:]

In [ ]:
train_au.shape, test_au.shape

In [ ]:
model_au = Prophet(seasonality_mode='multiplicative', interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True)
model_au.fit(train_au)

In [ ]:
horizon = 24
future = model_au.make_future_dataframe(periods=horizon)
forecast = model_au.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
ax = model_au.plot(forecast);

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(train_au['ds'], train_au['y'], label="Train")
plt.plot(test_au['ds'], test_au['y'], label="Test")
plt.plot(forecast[-horizon:]['ds'], forecast[-horizon:]['yhat'], label="predictions")
plt.legend(loc="best")
plt.show()

In [ ]:
model_au.plot_components(forecast);

In [ ]:
m_au = Prophet()
m_au.fit(dfp_au)

In [ ]:
future_au = m_au.make_future_dataframe(periods=365)
future_au.tail()

In [ ]:
forecast_au = m_au.predict(future_au)
forecast_au[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast_au

In [ ]:
f_yhat_au = forecast_au["yhat"]
f_AU_au = f_yhat_au[350:500]


e_y_au = dfp_au["y"]
e_AU_au = e_y_au[350:]


In [ ]:
e_AU_au = np.array(e_AU_au.values.tolist())

In [ ]:
expected_AU_au = e_AU_au
expected_AU_au

In [ ]:
f_AU_au = np.array(f_AU_au.values.tolist())

In [ ]:
forecast_AU_au = f_AU_au
forecast_AU_au

In [ ]:
fig2 = m_au.plot(forecast_au)


### Metrics

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast_AU_au - expected_AU_au)/np.abs(expected_AU_au))  # MAPE
    me = np.mean(forecast_AU_au - expected_AU_au)             # ME
    mae = np.mean(np.abs(forecast_AU_au - expected_AU_au))    # MAE
    mpe = np.mean((forecast_AU_au - expected_AU_au)/expected_AU_au)   # MPE
    rmse = np.mean((forecast_AU_au - expected_AU_au)**2)**.5  # RMSE
    corr = np.corrcoef(forecast_AU_au, expected_AU_au)[0,1]   # corr
    mins = np.amin(np.hstack([forecast_AU_au[:,None], 
                              expected_AU_au[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast_AU_au[:,None], 
                              expected_AU_au[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax                     
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(forecast_AU_au, expected_AU_au)

In [ ]:
cv_results_au = cross_validation(model=model_au, initial="343 days", horizon='30 days', period='7 days')

In [ ]:
cv_results_au

In [ ]:
dfp_au = performance_metrics(cv_results_au)
dfp_au

In [ ]:
plot_cross_validation_metric(cv_results_au, metric='mse');

## Prophet NewPayingUsers

In [ ]:
dfp_npu = pd.DataFrame(df, columns = ['Day\nDate', 'NewPayingUsers'])
dfp_npu

In [ ]:
dfp_npu.columns = ['ds', 'y']
dfp_npu.head()

In [ ]:
dfp_npu['ds'] = pd.to_datetime(dfp_npu['ds'])
dfp_npu

In [ ]:
plt.figure(figsize=(12, 8))
plt.grid(linestyle='-.')

plt.plot(dfp_npu['ds'], dfp_npu['y'], 'b')
plt.show()

In [ ]:
train_npu = dfp_npu.iloc[:440]
test_npu = dfp_npu.iloc[440:]

In [ ]:
train_npu.shape, test_npu.shape

In [ ]:
model_npu = Prophet(seasonality_mode='multiplicative', interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True)
model_npu.fit(train_npu)

In [ ]:
horizon = 36
future = model_npu.make_future_dataframe(periods=horizon)
forecast = model_npu.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
ax = model_npu.plot(forecast);

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(train_npu['ds'], train_npu['y'], label="Train")
plt.plot(test_npu['ds'], test_npu['y'], label="Test")
plt.plot(forecast[-horizon:]['ds'], forecast[-horizon:]['yhat'], label="predictions")
plt.legend(loc="best")
plt.show()

In [ ]:
model_npu.plot_components(forecast);

In [ ]:
cv_results_npu = cross_validation(model=model_npu, initial="343 days", horizon='30 days', period='60 days')

In [ ]:
cv_results_npu

In [ ]:
dfp_npu = performance_metrics(cv_results_npu)
dfp_npu

In [ ]:
plot_cross_validation_metric(cv_results_npu, metric='mse');

In [ ]:
plot_cross_validation_metric(cv_results_npu, metric='mae');

In [ ]:
plot_cross_validation_metric(cv_results_npu, metric='rmse');